In [1]:
import pandas as pd
import numpy as np
import re
import pickle

In [2]:
# open train text file
f = open(f'./covid_text.pckl','rb')
covid_data = pickle.load(f)
f.close()

In [3]:
f = open(f'./hashtag.pckl','rb')
hashtag = pickle.load(f)
f.close()

In [4]:
## cleaning the tweets
def clean_text(text):
    text = re.sub(r'@[A-Za-z0-9]+','',text) #remove @mention
    text = re.sub(r'#','',text) # remove the hashtag symbol
    text = re.sub(r'https?:\/\/\S+', '',text) #remove hyperlink
    text = re.sub(r'\n','',text) # remove \n 
    text = re.sub(r'\r','',text) # remove \r
    text = re.sub(r'[0-9]+','',text) #remove all the number
    text = re.sub(r'\W+', ' ', text) #remove special characters
    text = text.strip().lower()
    if len(text) != 0:
        return text
    else:
        return None
    
for i in range(len(covid_data)):
    for j in range(len(covid_data[i])):
        covid_data[i][j] = clean_text(covid_data[i][j])
    covid_data[i] = [x for x in covid_data[i] if x is not None]
        

In [5]:
# merge source tweeet and reply tweet together for test data    
covid_merge_events=[]
for event in covid_data:
    merge = ''
    for tweet in event:
        merge = merge + tweet
    covid_merge_events.append(merge)

In [6]:
covid_df = pd.DataFrame({'text':covid_merge_events})

nan_value = float("NaN")

covid_df.replace("", nan_value, inplace=True)

covid_df.dropna(axis=0 ,inplace=True)

covid_df.to_csv('covid.tsv', sep='\t',index=False)

In [7]:
covid_df.shape

(17012, 1)

In [8]:
df = pd.read_csv('covid_predict.csv')

In [9]:
df

,text,Predicted
0,according to the new york times warner bros wa...,0
1,hurricane hanna has made landfall in texas the...,0
2,monkeys on the loose in india with stolen coro...,0
3,let s not play blind that the work is fighting...,0
4,if trump felt comfortable having it here then ...,0
...,...,...
17007,i wonder how many lives could ve been saved if...,0
17008,the front page on th march the first line read...,0
17009,trump just completed the racism trifecta in a ...,1
17010,here are a few of my photographs from today s ...,0


In [10]:
df['text'][0]

'according to the new york times warner bros wanted to delay christopher nolan s tenet months ago due to the covid pandemic but nolan refused to listen to any discussions the studio kept the original release date to keep nolan happy'

In [11]:
df_nonrummor = df[df['Predicted'] == 0]

In [12]:
df_nonrummor.shape

(14914, 2)

In [13]:
df_rummor = df[df['Predicted'] == 1]

In [14]:
df_rummor.shape

(2098, 2)

In [15]:
# Installations
import sys
if 'google.colab' in sys.modules:
    !pip install emoji --upgrade
    !pip install pandas-profiling==2.*
    !pip install plotly==4.*
    !python -m spacy download en_core_web_lg
    !pip install pyldavis
    !pip install gensim
    !pip install chart_studio
    !pip install --upgrade autopep8

     |████████████████████████████████| 175 kB 7.6 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=62166d6a4d7b43047c10ce0e057b4d09c858ab30f466b35ad6dbf14e94f450eb
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
     |████████████████████████████████| 245 kB 6.9 MB/s 
     |████████████████████████████████| 690 kB 47.4 MB/s 
     |████████████████████████████████| 4.7 MB 37.9 MB/s 
     |████████████████████████████████| 102 kB 79.5 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 812 kB 54.1 MB/s 
     |████████████████████████████████| 38.1 MB 353 kB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=7972619485f774b7dfdfd8f03c3bd8b58dae5e72ac95af7301efd4f759e17927
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d250ba868768823940c3229e9950d201a26d0bd3ee8655

In [16]:
# Required Libraries

#Base and Cleaning 
import json
import requests
import pandas as pd
import numpy as np
import emoji
import regex
import re
import string
from collections import Counter

#Visualizations
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt 

import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls

#Natural Language Processing (NLP)
import spacy
import gensim
from spacy.tokenizer import Tokenizer
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS as SW
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)

In [17]:
rummors = df_rummor['text'].to_numpy()
non_rummors = df_nonrummor['text'].to_numpy()

In [18]:
from gensim.parsing.preprocessing import remove_stopwords

import nltk
nltk.download('punkt')
non_rummor_tokens = []
for rummor in non_rummors:
  rummor = remove_stopwords(rummor)
  token = nltk.word_tokenize(rummor)
  non_rummor_tokens.append(token)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [19]:
from nltk.stem import WordNetLemmatizer
 
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
for tokens in non_rummor_tokens:
  for i in range(len(tokens)):
    tokens[i] = lemmatizer.lemmatize(tokens[i])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [20]:
non_rummor_token_final =[]
for tokens in non_rummor_tokens:
  temp=[]
  for token in tokens:
    if len(token) >=3 and token not in temp:
      temp.append(token)
  non_rummor_token_final.append(temp) 

In [21]:
# Create a id2word dictionary
id2word = Dictionary(non_rummor_token_final)
print(len(id2word))

167618


In [22]:
# Filtering Extremes
id2word.filter_extremes(no_below=2, no_above=.99)
print(len(id2word))

41331


In [23]:
# Creating a corpus object 
corpus = [id2word.doc2bow(d) for d in non_rummor_token_final]

In [24]:
# Instantiating a Base LDA model 
base_model = LdaMulticore(corpus=corpus, num_topics=5, id2word=id2word, workers=12, passes=5)


In [25]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]


In [26]:
topics = [' '.join(t[0:10]) for t in words]

In [27]:
# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
covid coronavirus people trump death amp know country president virus

------ Topic 1 ------
covid coronavirus case state amp people death trump new number

------ Topic 2 ------
covid coronavirus people amp case pandemic death like time new

------ Topic 3 ------
coronavirus covid people virus amp like new trump think case

------ Topic 4 ------
coronavirus covid people amp pandemic know trump like death said



In [28]:
# Compute Perplexity
# a measure of how good the model is. lower the better
base_perplexity = base_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=base_model, texts=non_rummor_token_final, 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)


Perplexity:  -8.794070387078998

Coherence Score:  0.7174324268134512


In [29]:
import pyLDAvis.gensim_models

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [30]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(base_model, corpus, id2word)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.008053  0.007997       1        1  31.665107
0      0.033376 -0.010496       2        1  24.920046
3     -0.001088 -0.000142       3        1  23.775914
1     -0.000691  0.020860       4        1  14.878244
4     -0.023544 -0.018220       5        1   4.760689, topic_info=            Term          Freq         Total Category  logprob  loglift
26   coronavirus   9647.000000   9647.000000  Default  30.0000  30.0000
4          covid  10491.000000  10491.000000  Default  29.0000  29.0000
172       people   5330.000000   5330.000000  Default  28.0000  28.0000
15      pandemic   3255.000000   3255.000000  Default  27.0000  27.0000
72           amp   4423.000000   4423.000000  Default  26.0000  26.0000
..           ...           ...           ...      ...      ...      ...
69          work    107.715498   2015.998054   Topic5  -6.3073   0.1154
85          case    132.118275   3801.431640   Topic5  -6.1031  -0.3147
453      country    119.189343   2882.211720   Topic5  -6.2061  -0.1408
612     positive    104.100909   1950.494694   Topic5  -6.3415   0.1143
474   government    104.444893   2110.512312   Topic5  -6.3382   0.0388

[518 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
9470       1  0.802177  aboard
9470       2  0.114597  aboard
9470       3  0.057298  aboard
9470       4  0.057298  aboard
24338      3  0.785228   abort
...      ...       ...     ...
2069       1  0.539839   young
2069       2  0.153397   young
2069       3  0.174046   young
2069       4  0.100298   young
2069       5  0.035399   young

[1338 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 2, 5])

In [31]:
text = [' '.join(map(str, l)) for l in non_rummor_token_final]

In [32]:
vectorizer = CountVectorizer()
data_vectorized = vectorizer.fit_transform(text)

In [33]:
# Define Search Param
search_params = {'n_components': [10, 20, 30], 'learning_decay': [.5]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)
GridSearchCV(cv=None, error_score='raise',
             estimator=LatentDirichletAllocation(batch_size=128, 
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1, 
                                                 learning_decay=0.7, 
                                                 learning_method=None,
                                                 learning_offset=10.0, 
                                                 max_doc_update_iter=100, 
                                                 max_iter=10,
                                                 mean_change_tol=0.001, 
                                                 n_components=10, 
                                                 n_jobs=1,
                                                 perp_tol=0.1, 
                                                 random_state=None,
                                                 topic_word_prior=None, 
                                                 total_samples=1000000.0, 
                                                 verbose=0),
             n_jobs=1,
             param_grid={'n_topics': [10, 20, 30], 
                         'learning_decay': [0.5]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
             scoring=None, verbose=0)
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.5, 'n_components': 10}
Best Log Likelihood Score:  -3240720.6860533087
Model Perplexity:  13659.860988899432


In [34]:
model_final = LdaMulticore(corpus=corpus,
                       id2word=id2word,
                       num_topics=10,
                       decay=0.5)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [35]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in model_final.print_topics()]
topics = [' '.join(t[0:10]) for t in words]
# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
covid coronavirus country people amp positive need time think know

------ Topic 1 ------
covid coronavirus people need amp case know new time right

------ Topic 2 ------
coronavirus covid trump amp people pandemic think mask need work

------ Topic 3 ------
covid coronavirus people virus pandemic trump new amp like time

------ Topic 4 ------
covid coronavirus people new trump case like need health going

------ Topic 5 ------
covid coronavirus people amp like time pandemic american know trump

------ Topic 6 ------
coronavirus covid case people death trump new state day country

------ Topic 7 ------
coronavirus covid like people need force state day know thing

------ Topic 8 ------
covid trump coronavirus virus like amp president american state want

------ Topic 9 ------
covid coronavirus amp people death case new day virus like



In [36]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(model_final, corpus, id2word)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.043763  0.017761       1        1  22.004004
3      0.037069 -0.022294       2        1  13.026054
4      0.010557  0.005169       3        1  10.314419
8     -0.048547 -0.025577       4        1   9.802595
2     -0.006199  0.005375       5        1   9.304356
6      0.041952 -0.015822       6        1   8.652273
1      0.009963  0.026192       7        1   8.558221
5     -0.008260 -0.002821       8        1   7.651529
0     -0.051343  0.021722       9        1   6.490801
7     -0.028955 -0.009706      10        1   4.195749, topic_info=            Term          Freq         Total Category  logprob  loglift
26   coronavirus   9887.000000   9887.000000  Default  30.0000  30.0000
4          covid  10691.000000  10691.000000  Default  29.0000  29.0000
85          case   3847.000000   3847.000000  Default  28.0000  28.0000
172       people   5317.000000   5317.000000  Default  27.0000  27.0000
217        trump   3542.000000   3542.000000  Default  26.0000  26.0000
..           ...           ...           ...      ...      ...      ...
796        virus     77.115219   3464.565321  Topic10  -6.5152  -0.6339
217        trump     75.730294   3542.695027  Topic10  -6.5333  -0.6744
547     american     70.315302   2305.912979  Topic10  -6.6075  -0.3191
15      pandemic     72.252232   3320.825107  Topic10  -6.5804  -0.6567
148         life     69.637554   2424.764006  Topic10  -6.6172  -0.3791

[922 rows x 6 columns], token_table=       Topic      Freq                  Term
term                                        
34706      6  0.908352             _ferguson
14442      1  0.891240             _governor
14442      6  0.055702             _governor
27542      2  0.948462                   _ke
20831      9  0.898517              _samanta
...      ...       ...                   ...
13773      7  0.884881  yulindogmeatfestival
19979      6  0.955429                  zpac
29614      1  0.148183                   इनक
29614      9  0.740913                   इनक
22791      7  0.951864                   โคว

[2694 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 4, 5, 9, 3, 7, 2, 6, 1, 8])

Done